In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import arviz as az
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import pystan
%load_ext jupyterstan

PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
STAN_MODEL_PATH = os.path.join(PROJECT_ROOT_DIR, "stan_models")
STAN_DATA_PATH = os.path.join(PROJECT_ROOT_DIR,'data')

from stan_utils import *

 $E\left[\mathbf{Y}_{1} | \mathbf{Y}_{2}\right]=\boldsymbol{\mu}_{1}+\Omega_{12} \Omega_{22}^{-1}\left(\mathbf{Y}_{2}-\boldsymbol{\mu}_{2}\right) $
 
 $ \operatorname{Var}\left[\mathbf{Y}_{1} | \mathbf{Y}_{2}\right]=\Omega_{11}-\Omega_{12} \Omega_{22}^{-1} \Omega_{21} $

In [ ]:
%%stan prediction_model_knowpowerlocation
functions {
  matrix cov_matern32_tv(vector[] x, vector[] y, real alpha, real rho) {
    int N_x=size(x);
    int N_y=size(y);
    matrix[N_x, N_y] cov;
    for (i in 1:N_x){
      for (j in 1:N_y){
        cov[i,j] = square(alpha) * (1 + distance(x[i],y[j])/rho) * exp(-1*distance(x[i], y[j]) / rho);
      }
    }    
    return cov;
  }
  
  matrix cov_matern32(vector[] x,  real alpha, real rho) {
    int N=size(x);
    matrix[N, N] cov;
    for (i in 1:N){
      for (j in i:N){
          cov[i,j] = square(alpha) * (1 + distance(x[i],x[j])/rho) * exp(-1*distance(x[i], x[j]) / rho);
          cov[j,i] = cov[i, j];
      }
    }    
    return cov;
  }
  
  vector gp_pred_rng(vector[] x1, vector y2, vector[] x2, real gama, real fai, real sigma, real sigma_eta, real trans_power, vector trans_location) {
    int N1 = size(x1);
    int N2 = size(x2);
    vector[N1] y1;
    {
      matrix [N2, N2] L_K_2;
      vector[N2] K_div_y2;
      matrix [N2, N1] K_x2_x1;
      matrix [N2, N1] v_pred;
      vector [N1] y1_mu;
      vector [N1] mu1;
      vector [N2] mu2;
      matrix [N1, N1] y1_cov;
      matrix [N2, N2] K_2;
      K_2 = cov_matern32(x2, sigma, fai);
      
      for (n in 1:N2) {
        K_2[n, n] = K_2[n, n] + square(sigma_eta);
        mu2[n] = trans_power - 10 * gama * log10(distance(trans_location, x2[n]));
      }
      
      for (n in 1:N1) {
        mu1[n] = trans_power -10 * gama * log10(distance(trans_location, x1[n]));
      }
      
      
      L_K_2 = cholesky_decompose(K_2);
      K_div_y2 = mdivide_left_tri_low(L_K_2, y2 - mu2);
      K_div_y2 = mdivide_right_tri_low(K_div_y2', L_K_2)';
      K_x2_x1 = cov_matern32_tv(x2, x1, sigma, fai);
      y1_mu = mu1 + K_x2_x1' * K_div_y2;
      v_pred = mdivide_left_tri_low(L_K_2, K_x2_x1);
      y1_cov = cov_matern32(x1, sigma, fai) - v_pred' * v_pred;
      y1 = multi_normal_rng(y1_mu, y1_cov);
    }
    return y1;
  }
}

data {
  int<lower=1> N_pred; //需要预测的点个数
  vector[N_pred] pred_coord_x;// 每组数据有N个感知节点
  vector[N_pred] pred_coord_y;// 
  int<lower=1> N;
  vector[N] coord_x;
  vector[N] coord_y;
  vector[N] Y_observe;
  real trans_power; //信号源发送功率
  real trans_x; //信号源坐标
  real trans_y;
  
  int <lower=1> N_samples;
  vector[N_samples] gama; // 自由空间传播损耗因子
  vector[N_samples] fai;  // matern 协方差函数的距离 参数
  vector[N_samples] sigma; // 空间方差  
  vector[N_samples] sigma_eta; // 测量误差
 
}

transformed data {
  vector[2] points[N];
  vector[2] points_pred[N_pred];
  vector[2] trans_location;
  
  for (i in 1:N) {
    points[i][1] = coord_x[i];
    points[i][2] = coord_y[i];
  }
  
  for (i in 1:N_pred) {
    points_pred[i][1] = pred_coord_x[i];
    points_pred[i][2] = pred_coord_y[i];
  }
  
  trans_location[1] = trans_x;
  trans_location[2] = trans_y;
}

parameters {
}

transformed parameters {
}

model {   
}

generated quantities {
  matrix[N_pred, N_samples] pred_Y;
  for (i in 1:N_samples) {
    pred_Y[,i] = gp_pred_rng(points_pred, Y_observe, points, gama[i], fai[i], sigma[i], sigma_eta[i], trans_power, trans_location);
    }
  }

INFO:pystan:COMPILING THE C++ CODE FOR MODEL prediction_model_knowpowerlocation_ba3e783e245b86d3c2d85e9ea1d9ecad NOW.


Creating pystan model & assigning it to variable name "prediction_model_knowpowerlocation".
Stan options:
 {'model_name': 'prediction_model_knowpowerlocation', 'include_paths': None, 'boost_lib': None, 'eigen_lib': None, 'verbose': False, 'obfuscate_model_name': True}


In [3]:
StanModel_cache(prediction_model_knowpowerlocation,'prediction_model_knowpowerlocation')

Model cached as:prediction_model_knowpowerlocation.pkl
